In [2]:
import pandas as pd
import requests
import psycopg2
from sqlalchemy import create_engine

host = "localhost"
port = "5432"
database = "MDTM37DB"
username = "postgres"
password = "password"

In [ ]:
url = "https://api.sportradar.com/tennis/trial/v3/en/competitions.json?api_key=CREbOpUEx8KkSY2oHWgkYrIxA7QI5lIrZGrtJloh"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers).json()

category_list = response['competitions']

category_list

FETCH DATA FROM API AND CONVERTED INTO DATAFRAME FOR category_table

In [4]:
category_id = []
category_name = []
for value in category_list:
    category_id.append(value['category']['id'])
    category_name.append(value['category']['name'])

data = {'category_id': category_id, 'category_name': category_name}
category_table = pd.DataFrame(data)

print(category_table.duplicated().sum())

category_table = category_table.drop_duplicates()

print(category_table.duplicated().sum())

print(category_table.shape)

5862
0
(18, 2)


Inserting the data into postgreSQL

In [12]:
engine_string = f"postgresql://{username}:{password}@{host}:{port}/{database}"
engine = create_engine(engine_string)

category_table.to_sql('categories', engine, if_exists='append', index=False)

print("Data successfully pushed to PostgreSQL!")

Data successfully pushed to PostgreSQL!


FETCH DATA FROM API AND CONVERTED INTO DATAFRAME FOR competitions_table

In [17]:
competition_id = []
competition_name = []
parent_id = []
competition_type = []
gender = []
category_id = []

for value in category_list:
    competition_id.append(value.get('id'))
    competition_name.append(value.get('name'))
    parent_id.append(value.get('parent_id'))
    competition_type.append(value.get('type'))
    gender.append(value.get('gender'))
    category_id.append(value['category']['id'])

data = {'competition_id': competition_id, 'competition_name': competition_name, 'parent_id': parent_id, 'type': competition_type, 'gender': gender, 'category_id': category_id}
competitions_df = pd.DataFrame(data)

print(competitions_df.shape)

print(competitions_df.duplicated().sum())

competitions_df.drop_duplicates(subset=['competition_id'])

competitions_df

(5880, 6)
0


,competition_id,competition_name,parent_id,type,gender,category_id
0,sr:competition:620,Hopman Cup,None,mixed,mixed,sr:category:181
1,sr:competition:660,World Team Cup,None,mixed,men,sr:category:3
2,sr:competition:990,ATP Challenger Tour Finals,sr:competition:6239,singles,men,sr:category:72
3,sr:competition:1207,Championship International Series,None,singles,women,sr:category:6
4,sr:competition:2100,Davis Cup,None,mixed,men,sr:category:76
...,...,...,...,...,...,...
5875,sr:competition:45897,UTR Stockton M01,None,singles,men,sr:category:2516
5876,sr:competition:45899,UTR Spring W01,None,singles,women,sr:category:2517
5877,sr:competition:45913,UTR Bucuresti W01,None,singles,women,sr:category:2517
5878,sr:competition:45915,UTR Rungsted Kyst W01,None,singles,women,sr:category:2517


In [16]:
engine_string = f"postgresql://{username}:{password}@{host}:{port}/{database}"
engine = create_engine(engine_string)

competitions_df.to_sql('competitions', engine, if_exists='append', index=False)

print("Data successfully pushed to PostgreSQL!")

Data successfully pushed to PostgreSQL!


In [ ]:
url = "https://api.sportradar.com/tennis/trial/v3/en/complexes.json?api_key=CREbOpUEx8KkSY2oHWgkYrIxA7QI5lIrZGrtJloh"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers).json()

complexes_list = response['complexes']

complexes_list

FETCH DATA FROM API AND CONVERTED INTO DATAFRAME FOR complexes_table

In [8]:
complex_id = []
complex_name = []
for value in complexes_list:
    complex_id.append(value['id'])
    complex_name.append(value['name'])

data = {'complex_id': complex_id, 'complex_name': complex_name}
complexes_table = pd.DataFrame(data)

print(complexes_table.shape)

print(complexes_table.duplicated().sum())

complexes_table.drop_duplicates()

print(complexes_table.shape)

# complexes_table

(632, 2)
0
(632, 2)


In [26]:
engine_string = f"postgresql://{username}:{password}@{host}:{port}/{database}"
engine = create_engine(engine_string)

complexes_table.to_sql('complexes', engine, if_exists='append', index=False)

print("Data successfully pushed to PostgreSQL!")

Data successfully pushed to PostgreSQL!


FETCH DATA FROM API AND CONVERTED INTO DATAFRAME FOR venues_table

In [15]:
venue_id = []
venue_name = []
city_name = []
country_name = []
country_code = []
timezone = []
complex_id = []

count = []

for value in complexes_list:
    if 'venues' in value:
        for venue in value['venues']:
            venue_id.append(venue['id'])
            venue_name.append(venue['name'])
            city_name.append(venue['city_name'])
            country_name.append(venue['country_name'])
            country_code.append(venue['country_code'])
            timezone.append(venue['timezone'])
            complex_id.append(value['id'])
    else:
        count.append(1)


data = {'venue_id': venue_id, 'venue_name': venue_name, 'city_name':city_name, 'country_name': country_name, 'country_code': country_code, 'timezone': timezone, 'complex_id': complex_id}
venues_table = pd.DataFrame(data)

print(venues_table.shape)
print(venues_table.duplicated().sum())

print(venues_table.shape)
print(sum(count))


(3207, 7)
0
(3207, 7)
153


In [39]:
engine_string = f"postgresql://{username}:{password}@{host}:{port}/{database}"
engine = create_engine(engine_string)

venues_table.to_sql('venues', engine, if_exists='append', index=False)

print("Data successfully pushed to PostgreSQL!")

Data successfully pushed to PostgreSQL!


In [ ]:
url = "https://api.sportradar.com/tennis/trial/v3/en/double_competitors_rankings.json?api_key=CREbOpUEx8KkSY2oHWgkYrIxA7QI5lIrZGrtJloh"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers).json()

doubles_competitor_rankings = response['rankings'][0]['competitor_rankings']

doubles_competitor_rankings

FETCH DATA FROM API AND CONVERTED INTO DATAFRAME FOR competitors_table

In [14]:
competitor_id = []
name = []
country = []
country_code = []
abbreviation = []

for value in doubles_competitor_rankings:
    competitor_id.append(value['competitor'].get('id'))
    name.append(value['competitor'].get('name'))
    country.append(value['competitor'].get('country'))
    abbreviation.append(value['competitor'].get('abbreviation'))
    
    if value['competitor'].get('country') == 'Neutral':
        country_code.append('UNK')  
    else:
        country_code.append(value['competitor'].get('country_code'))

data = {'competitor_id': competitor_id, 'name': name, 'country': country, 'country_code': country_code, 'abbreviation': abbreviation}
competitors_table = pd.DataFrame(data)

print(competitors_table.shape)

print(competitors_table.duplicated().sum())

competitors_table


(500, 5)
0


,competitor_id,name,country,country_code,abbreviation
0,sr:competitor:49363,"Pavic, Mate",Croatia,HRV,PAV
1,sr:competitor:51836,"Arevalo-Gonzalez, Marcelo",El Salvador,SLV,ARE
2,sr:competitor:637970,"Patten, Henry",Great Britain,GBR,PAT
3,sr:competitor:14898,"Heliovaara, Harri",Finland,FIN,HEL
4,sr:competitor:87690,"Thompson, Jordan",Australia,AUS,THO
...,...,...,...,...,...
495,sr:competitor:801374,"Gea, Arthur",France,FRA,GEA
496,sr:competitor:809324,"Pacheco Mendez, Rodrigo",Mexico,MEX,PAC
497,sr:competitor:434058,"Boscardin Dias, Pedro",Brazil,BRA,BOS
498,sr:competitor:908699,"Bilardo, Jacopo",Italy,ITA,BIL


In [65]:
engine_string = f"postgresql://{username}:{password}@{host}:{port}/{database}"
engine = create_engine(engine_string)

competitors_table.to_sql('competitors', engine, if_exists='append', index=False)

print("Data successfully pushed to PostgreSQL!")

Data successfully pushed to PostgreSQL!


FETCH DATA FROM API AND CONVERTED INTO DATAFRAME FOR Competitor_Rankings_table

In [13]:
rank = []
movement = []
points = []
competitions_played = []
competitor_id = []

for value in doubles_competitor_rankings:
    rank.append(value['rank'])
    movement.append(value['movement'])
    points.append(value['points'])
    competitions_played.append(value['competitions_played'])
    competitor_id.append(value['competitor']['id'])

data = {'rank': rank, 'movement': movement, 'points': points, 'competitions_played': competitions_played, 'competitor_id': competitor_id}
Competitor_Rankings_table = pd.DataFrame(data)

print(Competitor_Rankings_table.shape)

print(Competitor_Rankings_table.duplicated().sum())


(500, 5)
0


,rank,movement,points,competitions_played,competitor_id
0,1,0,7530,22,sr:competitor:49363
1,1,0,7530,22,sr:competitor:51836
2,3,10,7075,31,sr:competitor:637970
3,4,10,7055,28,sr:competitor:14898
4,5,-2,6475,21,sr:competitor:87690
...,...,...,...,...,...
495,496,3,106,11,sr:competitor:801374
496,497,63,106,12,sr:competitor:809324
497,498,38,106,18,sr:competitor:434058
498,499,2,106,18,sr:competitor:908699


In [ ]:
engine_string = f"postgresql://{username}:{password}@{host}:{port}/{database}"
engine = create_engine(engine_string)

Competitor_Rankings_table.to_sql('competitor_rankings', engine, if_exists='append', index=False)

print("Data successfully pushed to PostgreSQL!")

Data successfully pushed to PostgreSQL!


In [ ]:
engine_string = f"postgresql://{username}:{password}@{host}:{port}/{database}"
engine = create_engine(engine_string)

Competitor_Rankings_query = "SELECT * FROM competitor_rankings"
Competitor_Rankings = pd.read_sql_query(Competitor_Rankings_query, engine)

Competitor_Rankings

# Competitor_Rankings.shape
